In [1]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
SIG_X = np.array([[0, 1], [1, 0]])
SIG_Y = np.array([[0, -1j], [1j, 0]])
SIG_Z = np.array([[1, 0], [0, -1]])

In [3]:
# Here assuming that a = 1 for convenience
def build_ham(kx, ky, kz, M):
    mat = np.zeros((2, 2), dtype=complex)
    mat += -np.sin(kx) * SIG_X
    mat += -np.sin(ky) * SIG_Y
    mat += (M - np.cos(kx) - np.cos(ky) - np.cos(kz)) * SIG_Z
    return mat

In [4]:
N = 30
ks = np.linspace(-np.pi, np.pi, N+1, endpoint=True)

### Item 1b

In [ ]:
# Predefs

M = 2.866
energies = np.zeros((len(ks), len(ks), len(ks), 2))

# Calculate the energy spctrum

for i1, kx in enumerate(ks):
    for i2, ky in enumerate(ks):
        for i3, kz in enumerate(ks):
            ham = build_ham(kx, ky, kz, M)
            sols = np.linalg.eigh(ham)
            energies[i1, i2, i3, :] = sols[0]

# Plot the spectrum for constant M, kz

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X, Y = np.meshgrid(ks, ks)
kz_index = 12
ax.plot_surface(X, Y, energies[:, :, kz_index, 0], cmap=cm.coolwarm,
                linewidth=0, antialiased=False)
ax.plot_surface(X, Y, energies[:, :, kz_index, 1], cmap=cm.coolwarm,
                linewidth=0, antialiased=False)
ax.title.set_text(f"Band Diagram for M=2, kz=-π/6")
ax.set_xlabel("kx")
ax.set_ylabel("ky")
ax.set_zlabel("energy")
fig.tight_layout()
plt.show()

### Items 1c + 1d

In [ ]:
# Predefs

Ms = (-3, -1, 1, 3)
N = 100
ks = np.linspace(-np.pi, np.pi, N, endpoint=False)
pi2 = np.pi / 2 # We set k_z = pi / 2 in order to only have M' present

# Link and Curvature Calculations

def link(eigv1, eigv2):
    return np.vdot(eigv1, eigv2) / np.abs(np.vdot(eigv1, eigv2))

def local_link(eigvs, ix: int, iy: int, dir: int):
    if dir % 2 == 0:
        ix_o = (ix + (1 - dir)) % N
        iy_o = iy
    else:
        ix_o = ix
        iy_o = (iy + (2 - dir)) % N
    return np.angle(link(eigvs[ix, iy, :], eigvs[ix_o, iy_o, :]), deg=False)


def local_curv(eigvs, ix: int, iy: int):
    link_sum = 0
    for dir in (0, 1, 2, 3):
        link_sum += local_link(eigvs, ix, iy, dir)
        if dir % 2 == 0:
            ix = (ix + (1 - dir)) % N
        else:
            iy = (iy + (2 - dir)) % N
    return link_sum

def local_curv(eigvs, ix: int, iy: int):
    ix1, iy1 = (ix + 1) % N, iy
    ix2, iy2 = ix1, (iy + 1) % N
    ix3, iy3 = ix, (iy + 1) % N

    U1 = link(eigvs[ix, iy], eigvs[ix1, iy1])
    U2 = link(eigvs[ix1, iy1], eigvs[ix2, iy2])
    U3 = link(eigvs[ix2, iy2], eigvs[ix3, iy3])
    U4 = link(eigvs[ix3, iy3], eigvs[ix, iy])

    return np.angle(U1 * U2 * U3 * U4)

fig, axs = plt.subplots(2, 2, subplot_kw={'projection': '3d'})#, constrained_layout=True)
fig.tight_layout(pad=1)
fig.suptitle("Berry Curvature")
fig.subplots_adjust(top=0.88)

# Calculate the Chern Number

for iM, M in enumerate(Ms):
    eigvs = np.zeros((len(ks), len(ks), 2), dtype=complex)

    for ix, kx in enumerate(ks):
        for iy, ky in enumerate(ks):
            hamiltonian = build_ham(kx, ky, pi2, M)
            # According to documentation, the eigenvectors are sorted by eigenvalue, so the first one is the - band vector
            minus_eigv = np.linalg.eigh(hamiltonian)[1][:, 0]
            eigvs[ix, iy, :] = minus_eigv

    curv = 0
    curv_grid = np.zeros((N, N), dtype=float)
    KX, KY = np.meshgrid(ks, ks, indexing="ij")

    for ix, kx in enumerate(ks):
        for iy, ky in enumerate(ks):
            curv_grid[ix, iy] = local_curv(eigvs, ix, iy)
            curv += curv_grid[ix, iy]

    # Log / Plot

    print(f"For M' = {M}, the Chern number of the + band is {np.round(curv / 2 / np.pi)}")

    ax = axs[iM // 2, iM % 2]
    surf = ax.plot_surface(KX, KY, curv_grid, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_title(f"M' = {M}", pad=-20)
    ax.set_xlabel("kx")
    ax.set_ylabel("ky")

plt.show()

In [ ]:
Ms = [1.99999999]

fig, ax = plt.subplots(subplot_kw={'projection': '3d'})#, constrained_layout=True)
fig.tight_layout(pad=1)
fig.suptitle(f"Berry Curvature for M'=2")
fig.subplots_adjust(top=0.88)


for iM, M in enumerate(Ms):
    eigvs = np.zeros((len(ks), len(ks), 2), dtype=complex)

    for ix, kx in enumerate(ks):
        for iy, ky in enumerate(ks):
            hamiltonian = build_ham(kx, ky, pi2, M)
            # According to documentation, the eigenvectors are sorted by eigenvalue, so the first one is the - band vector
            minus_eigv = np.linalg.eigh(hamiltonian)[1][:, 0]
            eigvs[ix, iy, :] = minus_eigv

    curv = 0
    curv_grid = np.zeros((N, N), dtype=float)
    KX, KY = np.meshgrid(ks, ks, indexing="ij")

    for ix, kx in enumerate(ks):
        for iy, ky in enumerate(ks):
            curv_grid[ix, iy] = local_curv(eigvs, ix, iy)
            curv += curv_grid[ix, iy]

    # Log / Plot

    print(f"For M' = {M}, the Chern number of the + band is {np.round(curv / 2 / np.pi)}")

    surf = ax.plot_surface(KX, KY, curv_grid, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_xlabel("kx")
    ax.set_ylabel("ky")

plt.show()

### Item 1e

In [5]:
# Matrix preparation auxiliary functions

def prep_diags(N, d):
    if d == 0:
        return np.eye(N, dtype=complex)
    else:
        mat = np.zeros((N, N), dtype=complex)
        if d == 1:
            for i in range(N - 1):
                mat[i + 1][i] = 1
        else: # d == - 1
            for i in range(N - 1):
                mat[i][i + 1] = 1
        return mat

def mat2quart(mat, q):
    N = mat.shape[0]
    board = np.zeros((2 * N, 2 * N), dtype=complex)
    if q == 0:
        board[:N, :N] = mat
    elif q == 1:
        board[:N, N:] = mat
    elif q == 2:
        board[N:, :N] = mat
    else: # q == 3
        board[N:, N:] = mat
    return board

def dot_pauli(mat, pauli_idx):
    board = np.zeros((2 * mat.shape[0], 2 * mat.shape[0]), dtype=complex)
    if pauli_idx == 'x':
        board += mat2quart(mat, 1) + mat2quart(mat, 2)
    elif pauli_idx == 'y':
        board += -1j * mat2quart(mat, 1) + 1j * mat2quart(mat, 2)
    elif pauli_idx == 'z':
        board += mat2quart(mat, 0) - mat2quart(mat, 3)
    return np.matrix(board)

def fourier_x_ham(N, M, ky, kz):
    return -1 / 2j * dot_pauli((prep_diags(N, 1) - prep_diags(N, -1)), 'x') + \
           -np.sin(ky) * dot_pauli(prep_diags(N, 0), 'y') + \
           dot_pauli(((M - np.cos(ky) - np.cos(kz)) * prep_diags(N, 0) - 1 / 2 * (prep_diags(N, 1) + prep_diags(N, -1))), 'z')

def fourier_y_ham(N, M, kx, kz):
    return -np.sin(kx) * dot_pauli(prep_diags(N, 0), 'x') + \
           -1 / 2j * dot_pauli((prep_diags(N, 1) - prep_diags(N, -1)), 'y') + \
           dot_pauli(((M - np.cos(kx) - np.cos(kz)) * prep_diags(N, 0) - 1 / 2 * (prep_diags(N, 1) + prep_diags(N, -1))), 'z')

def fourier_z_ham(N, M, kx, ky):
    return -np.sin(kx) * dot_pauli(prep_diags(N, 0), 'x') + \
           -np.sin(ky) * dot_pauli(prep_diags(N, 0), 'y') + \
           dot_pauli(((M - np.cos(kx) - np.cos(ky)) * prep_diags(N, 0) - 1 / 2 * (prep_diags(N, 1) + prep_diags(N, -1))), 'z')

In [12]:
# Predef

M = 1.5
N = 20
ks = np.linspace(-np.pi, np.pi, N, endpoint=False)

eigvls = np.zeros((len(ks), len(ks), 2 * N), dtype=complex)
eigvs = np.zeros((len(ks), len(ks), 2 * N, 2 * N), dtype=complex)

for ix, kx in enumerate(ks):
    for iy, ky in enumerate(ks):
        hamiltonian = fourier_z_ham(N, M, kx, ky)
        # According to documentation, the eigenvectors are sorted by eigenvalue, so the last one is the + band vector
        eigvls[ix, iy, :], cur_eigvs = np.linalg.eigh(hamiltonian)
        eigvs[ix, iy, :, :] = cur_eigvs

X, Y = np.meshgrid(np.arange(0, 2 * N, 1),
                         np.arange(0, N, 1))
X, Y = np.meshgrid(np.arange(0, N, 1), ks)

fig, axs = plt.subplots(1, 2, subplot_kw={"projection": "3d"})
fig.suptitle("Eigenfunctions for M=1.5, ky=π/4")
surf = axs[0].plot_surface(X, Y, np.abs(eigvs[:, 10, :N, N]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0].set_xlabel('z (site)')
axs[0].set_ylabel('kx')
axs[0].title.set_text('Sublattice A')

surf = axs[1].plot_surface(X, Y, np.abs(eigvs[:, 10, :N, N]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1].set_xlabel('z (site)')
axs[1].set_ylabel('kx')
axs[1].title.set_text(f"Sublattice B")
plt.show()

In [10]:
for band in range(2 * N):
    print(eigvls[:, 10, band])
    plt.plot(ks, eigvls[:, 10, band])
plt.title(f"Band Structure for M=1.5")
plt.ylabel("Energy [arb. units]")
plt.xlabel("ky")
plt.show()

[-2.48883083+0.j -2.45937833+0.j -2.37183391+0.j -2.22864148+0.j
 -2.03390395+0.j -1.79349302+0.j -1.51541056+0.j -1.34670363+0.j
 -1.42474576+0.j -1.47267344+0.j -1.48883083+0.j -1.47267344+0.j
 -1.42474576+0.j -1.34670363+0.j -1.51541056+0.j -1.79349302+0.j
 -2.03390395+0.j -2.22864148+0.j -2.37183391+0.j -2.45937833+0.j]
[-2.45557281+0.j -2.42638748+0.j -2.33962785+0.j -2.19768529+0.j
 -2.0045662 +0.j -1.76598193+0.j -1.48966396+0.j -1.32026684+0.j
 -1.39451743+0.j -1.44017275+0.j -1.45557281+0.j -1.44017275+0.j
 -1.39451743+0.j -1.32026684+0.j -1.48966396+0.j -1.76598193+0.j
 -2.0045662 +0.j -2.19768529+0.j -2.33962785+0.j -2.42638748+0.j]
[-2.40096887+0.j -2.37223838+0.j -2.28681635+0.j -2.14700989+0.j
 -1.95667068+0.j -1.72125355+0.j -1.44805642+0.j -1.2775536 +0.j
 -1.34519786+0.j -1.3868901 +0.j -1.40096887+0.j -1.3868901 +0.j
 -1.34519786+0.j -1.2775536 +0.j -1.44805642+0.j -1.72125355+0.j
 -1.95667068+0.j -2.14700989+0.j -2.28681635+0.j -2.37223838+0.j]
[-2.32623877+0.j -2.29